In [ ]:
import os
from typing import List

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM



DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

SUMMARY_MODEL_NAME = "gogamza/kobart-summarization"   # KoBART 요약 모델




def load_summarizer(model_name: str = SUMMARY_MODEL_NAME):
    """
    요약용 모델/토크나이저 로드
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    if DEVICE.type == "cuda":
        model.half()

    model.to(DEVICE)
    model.eval()
    return tokenizer, model


def analyze_and_filter_by_token_length(
    df: pd.DataFrame,
    tokenizer,
    content_col: str = "content",
    min_tokens: int = 300,
) -> pd.DataFrame:
    """
    1) 각 문서에 대해 KoBART 토크나이저로 토큰 길이를 계산
    2) 전체 분포/평균/최솟값/최댓값/분위수 출력
    3) 토큰 수가 min_tokens 이하인 행은 버린 뒤 필터링된 df 반환
    """
    texts = df[content_col].fillna("").tolist()
    token_lengths: List[int] = []

    print(f"[토큰 길이 분석] 총 {len(texts)}개 문서에 대해 토큰 길이 계산 중...")

    for i, text in enumerate(texts):
        enc = tokenizer(
            str(text),
            add_special_tokens=True,
            truncation=False,          # 여기서는 자르지 않고 전체 길이 측정
            return_attention_mask=False,
            return_tensors=None,
        )
        input_ids = enc["input_ids"]

        if isinstance(input_ids[0], list):
            input_ids = input_ids[0]
        token_lengths.append(len(input_ids))

        if i % 1000 == 0:
            print(f"  - {i}/{len(texts)} 처리 중...")

    df = df.copy()
    df["token_len"] = token_lengths

    lengths = pd.Series(token_lengths)
    print("\n[토큰 길이 통계]")
    print(f"  문서 수: {len(lengths)}")
    print(f"  최소 길이: {lengths.min()}")
    print(f"  최대 길이: {lengths.max()}")
    print(f"  평균 길이: {lengths.mean():.2f}")
    print(f"  중앙값(median): {lengths.median():.2f}")
    print(f"  25% 분위수: {lengths.quantile(0.25):.2f}")
    print(f"  75% 분위수: {lengths.quantile(0.75):.2f}")

    return df



def chunk_by_tokens(
    text: str,
    tokenizer,
    max_tokens: int = 1024,
) -> List[List[int]]:
    """
    긴 텍스트를 '토큰 개수' 기준으로 나누는 함수.
    """
    text = str(text)

    encoded = tokenizer(
        text,
        add_special_tokens=True,
        return_attention_mask=False,
        truncation=False,
        return_tensors=None,
    )["input_ids"]


    if isinstance(encoded[0], list):
        encoded = encoded[0]

    chunks: List[List[int]] = []
    n_tokens = len(encoded)

    for i in range(0, n_tokens, max_tokens):
        chunk_ids = encoded[i:i + max_tokens]
        chunks.append(chunk_ids)

    return chunks


@torch.no_grad()
def summarize_long_text(
    text: str,
    tokenizer,
    model,
    max_input_tokens: int = 1024,   # KoBART가 한 번에 받을 수 있는 최대 토큰 수
    max_summary_length: int = 512,  # 최종 요약 최대 길이
    num_beams: int = 2,
    min_summary_tokens: int = 256,  # 최소 요약 토큰 수 (부분 요약 + 최종 요약 모두)
) -> str:
    """
    긴 문서를 처리하기 위한 계층적 요약 함수 (토큰 단위 chunking).
    - total_tokens <= max_input_tokens 인 경우: 바로 generate (min_length 적용)
    - 초과인 경우:
        1) chunk 단위로 부분 요약 (각 chunk 요약에도 min_length 적용)
        2) 부분 요약들을 합쳐 다시 한 번 최종 요약 (여기도 min_length 적용)
    """
    if not isinstance(text, str) or text.strip() == "":
        return ""

    # 전체를 토큰화해서 토큰 개수 확인 (truncation=False)
    enc = tokenizer(
        text,
        add_special_tokens=True,
        return_attention_mask=False,
        truncation=False,
        return_tensors=None,
    )
    input_ids = enc["input_ids"]
    if isinstance(input_ids[0], list):
        input_ids = input_ids[0]
    total_tokens = len(input_ids)

    use_amp = (DEVICE.type == "cuda")

    # 1) max_input_tokens 이하면 바로 최종 요약
    if total_tokens <= max_input_tokens:
        enc2 = tokenizer(
            text,
            padding=True,
            truncation=True,
            max_length=max_input_tokens,
            return_tensors="pt",
        ).to(DEVICE)

        with torch.amp.autocast("cuda", enabled=use_amp):
            summary_ids = model.generate(
                **enc2,
                max_length=max_summary_length,
                min_length=min_summary_tokens,   # 
                num_beams=num_beams,
                early_stopping=True,
                no_repeat_ngram_size=3,
            )
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary.strip()

    token_chunks = chunk_by_tokens(
        text=text,
        tokenizer=tokenizer,
        max_tokens=max_input_tokens,
    )

    # 여러 chunk를 한 번에 batch로 요약 (부분 요약에도 min_length 적용)
    pad_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0
    input_ids_list = [torch.tensor(chunk, dtype=torch.long) for chunk in token_chunks]
    input_ids_padded = torch.nn.utils.rnn.pad_sequence(
        input_ids_list, batch_first=True, padding_value=pad_id
    ).to(DEVICE)
    attention_mask = (input_ids_padded != pad_id).long()

    with torch.amp.autocast("cuda", enabled=use_amp):
        summary_ids_batch = model.generate(
            input_ids=input_ids_padded,
            attention_mask=attention_mask,
            max_length=max_summary_length,
            min_length=min_summary_tokens,    
            num_beams=num_beams,
            early_stopping=True,
            no_repeat_ngram_size=3,
        )

    partial_summaries: List[str] = []
    for sid in summary_ids_batch:
        s = tokenizer.decode(sid, skip_special_tokens=True)
        partial_summaries.append(s.strip())

    combined = "\n".join(partial_summaries)

    enc3 = tokenizer(
        combined,
        padding=True,
        truncation=True,
        max_length=max_input_tokens,
        return_tensors="pt",
    ).to(DEVICE)

    with torch.amp.autocast("cuda", enabled=use_amp):
        final_ids = model.generate(
            **enc3,
            max_length=max_summary_length,
            min_length=min_summary_tokens,  
            num_beams=num_beams,
            early_stopping=True,
            no_repeat_ngram_size=3,
        )
    final_summary = tokenizer.decode(final_ids[0], skip_special_tokens=True)
    return final_summary.strip()



def summarize_excel(
    input_path: str,
    output_path: str,
    content_col: str = "content",
    summary_col: str = "summary",
    max_input_tokens: int = 1024,
    max_summary_length: int = 512,
    min_tokens: int = 300,
    num_beams: int = 2,
    min_summary_tokens: int = 256,
):
    """
    엑셀 파일을 읽어서 **기사(article_id) 단위로** content_col을 요약하고
    summary_col에 저장한 뒤 output_path로 저장.

    - 동일 article_id는 content가 같으므로 한 번만 요약
    - 그 summary를 같은 article_id의 모든 행에 채워 넣음
    - referrer 기반 다운샘플링은 하지 않음
    """
    print(f"[요약] 입력 파일: {input_path}")
    df = pd.read_excel(input_path)

    needed_cols = [content_col, "article_id"]
    for c in needed_cols:
        if c not in df.columns:
            raise ValueError(f"필수 컬럼 '{c}' 이(가) 엑셀에 없습니다.")

    df = df.copy()

    tokenizer, model = load_summarizer()


    article_df = (
        df[["article_id", content_col]]
        .drop_duplicates(subset=["article_id"])
        .reset_index(drop=True)
    )
    print(f"\n[INFO] 기사 단위 unique 개수: {len(article_df)}")


    article_df = analyze_and_filter_by_token_length(
        article_df,
        tokenizer=tokenizer,
        content_col=content_col,
        min_tokens=min_tokens,
    )

    # # 토큰 필터링에서 살아남은 article_id만 사용
    # valid_article_ids = set(article_df["article_id"].tolist())
    # df = df[df["article_id"].isin(valid_article_ids)].reset_index(drop=True)

    texts = article_df[content_col].fillna("").tolist()
    article_ids = article_df["article_id"].tolist()
    summaries: List[str] = []

    print(f"\n[요약] 필터링 후 {len(texts)}개 '기사' 요약 시작...")

    for i, text in enumerate(texts):
        summary = summarize_long_text(
            text,
            tokenizer,
            model,
            max_input_tokens=max_input_tokens,
            max_summary_length=max_summary_length,
            num_beams=num_beams,
            min_summary_tokens=min_summary_tokens,
        )
        summaries.append(summary)

        if i % 50 == 0:
            print(f"  - {i}/{len(texts)} 개 기사 처리 중...")


    summary_df = pd.DataFrame({
        "article_id": article_ids,
        summary_col: summaries,
    })

    df = df.merge(summary_df, on="article_id", how="left")

    df.to_excel(output_path, index=False)
    print(f"[요약] 완료! 기사 단위 요약 결과를 '{summary_col}' 컬럼으로 {output_path} 에 저장했습니다.")



if __name__ == "__main__":
    original_excel = "../AI_model/신문 데이터/news_merged_grouped_final.xlsx"
    summarized_excel = "../AI_model/신문 데이터/news_merged_grouped_final_summary.xlsx"

    summarize_excel(
        input_path=original_excel,
        output_path=summarized_excel,
        content_col="content",   # 원본 기사 본문 컬럼명
        summary_col="summary",   # 요약 내용이 들어갈 컬럼명
        max_input_tokens=1024,   # KoBART 최대 토큰 길이
        max_summary_length=512,
        min_tokens=300,          # 입력 토큰 수 <= 300인 기사는 버림
        num_beams=2,
        min_summary_tokens=256,  
    )


DEVICE: cuda
[요약] 입력 파일: ../AI_model/신문 데이터/news_merged_grouped_final.xlsx


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.



[INFO] 기사 단위 unique 개수: 1729
[토큰 길이 분석] 총 1729개 문서에 대해 토큰 길이 계산 중...
  - 0/1729 처리 중...
  - 1000/1729 처리 중...

[토큰 길이 통계]
  문서 수: 1729
  최소 길이: 2
  최대 길이: 13128
  평균 길이: 2245.12
  중앙값(median): 2319.00
  25% 분위수: 1778.00
  75% 분위수: 2795.00

[요약] 필터링 후 1729개 '기사' 요약 시작...
  - 0/1729 개 기사 처리 중...
  - 50/1729 개 기사 처리 중...
  - 100/1729 개 기사 처리 중...
  - 150/1729 개 기사 처리 중...
  - 200/1729 개 기사 처리 중...
  - 250/1729 개 기사 처리 중...
  - 300/1729 개 기사 처리 중...
  - 350/1729 개 기사 처리 중...
  - 400/1729 개 기사 처리 중...
  - 450/1729 개 기사 처리 중...
  - 500/1729 개 기사 처리 중...
  - 550/1729 개 기사 처리 중...
  - 600/1729 개 기사 처리 중...
  - 650/1729 개 기사 처리 중...
  - 700/1729 개 기사 처리 중...
  - 750/1729 개 기사 처리 중...
  - 800/1729 개 기사 처리 중...
  - 850/1729 개 기사 처리 중...
  - 900/1729 개 기사 처리 중...
  - 950/1729 개 기사 처리 중...
  - 1000/1729 개 기사 처리 중...
  - 1050/1729 개 기사 처리 중...
  - 1100/1729 개 기사 처리 중...
  - 1150/1729 개 기사 처리 중...
  - 1200/1729 개 기사 처리 중...
  - 1250/1729 개 기사 처리 중...
  - 1300/1729 개 기사 처리 중...
  - 1350/1729 개 기사 처리 

In [2]:
import pandas as pd
from transformers import AutoTokenizer

# 요약된 엑셀 위치
excel_path = "../AI_model/신문 데이터/news_merged_grouped_balanced_summary.xlsx"

# KoBART 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("gogamza/kobart-summarization")

# 엑셀 읽기
df = pd.read_excel(excel_path)

# summary 컬럼 존재 확인
if "summary" not in df.columns:
    raise ValueError("엑셀에 'summary' 컬럼이 없습니다.")

summaries = df["summary"].fillna("").tolist()

token_lengths = []

for i, text in enumerate(summaries):
    enc = tokenizer(
        str(text),
        add_special_tokens=True,
        truncation=False,
        return_attention_mask=False,
        return_tensors=None,
    )

    ids = enc["input_ids"]
    if isinstance(ids[0], list):
        ids = ids[0]

    token_lengths.append(len(ids))

print("문서 수:", len(token_lengths))
print("최소 토큰:", min(token_lengths))
print("최대 토큰:", max(token_lengths))
print("평균 토큰:", sum(token_lengths) / len(token_lengths))
print("중앙값:", sorted(token_lengths)[len(token_lengths) // 2])


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


문서 수: 15322
최소 토큰: 6
최대 토큰: 255
평균 토큰: 107.78756037070879
중앙값: 82
